In [68]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [33]:
!pip install beautifulsoup4 

     |████████████████████████████████| 122kB 21.8MB/s eta 0:00:01


In [82]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq

## PART 1 cluster the neighborhoods in Toronto

In [138]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [139]:
# parse data from the html into a beautifulsoup object
soup = bsoup(data, 'html.parser')

In [140]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [141]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [142]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [143]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [144]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [145]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [146]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,"Wexford, Maryvale"
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har..."


In [147]:
toronto_df_grouped.shape

(180, 3)

## End of Part 1

## PART 2

In [148]:
# Read the cvs file and convert it to a dataframe
url='https://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [150]:
#rename the column "Postal Code" to "PostalCode"

df_pcodes.columns = ['PostalCode', 'Latitude', 'Longitude']
df_pcodes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [153]:
df=toronto_df_grouped.sort_values('PostalCode')
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [154]:
neighborhoods=pd.merge(df,df_pcodes, how='right', on = 'PostalCode')
neighborhoods.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,NaN,NaN,43.806686,-79.194353
1,M1C,NaN,NaN,43.784535,-79.160497
2,M1E,NaN,NaN,43.763573,-79.188711
3,M1G,NaN,NaN,43.770992,-79.216917
4,M1H,NaN,NaN,43.773136,-79.239476
5,M1J,NaN,NaN,43.744734,-79.239476
6,M1K,NaN,NaN,43.727929,-79.262029
7,M1L,NaN,NaN,43.711112,-79.284577
8,M1M,NaN,NaN,43.716316,-79.239476
9,M1N,NaN,NaN,43.692657,-79.264848


In [155]:
neighborhoods.shape

(103, 5)

## End of PART 2

## PART 3

In [158]:
#focus on a subset of all the Borough that contain the word "Toronto"
toronto_data= neighborhoods[neighborhoods['Borough'].str.contains('Toronto', na = True)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,NaN,NaN,43.806686,-79.194353
1,M1C,NaN,NaN,43.784535,-79.160497
2,M1E,NaN,NaN,43.763573,-79.188711
3,M1G,NaN,NaN,43.770992,-79.216917
4,M1H,NaN,NaN,43.773136,-79.239476


In [159]:
toronto_data.shape

(103, 5)

In [160]:
# From google : the latitud and longitud for Toronto, as follows: 43.6532° N, 79.3832° W
latitude = 43.6532
longitude= -79.3832

In [162]:
#create a map of Toronto with Boroughs that cointain the word Toronto
# create map of TORONTO using latitude and longitude values above:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto